In [ ]:
import pywwt; print(pywwt.__version__)

In [ ]:
from pywwt.jupyter import WWTJupyterWidget
from astropy.coordinates import SkyCoord, concatenate
from astropy import units as u
from astropy.table import Table, Column
from astropy.time import Time, TimeDelta
from datetime import datetime
import numpy as np
import pytz

Run `to_test.py`

In [ ]:
from astropy.table import Table, Column
from astropy.time import Time, TimeDelta
import numpy as np
from io import StringIO
from base64 import b64encode

In [ ]:
# download earthquake table
EARTHQUAKES = 'https://worldwidetelescope.github.io/pywwt/data/earthquakes_2010.csv'
q_table = Table.read(EARTHQUAKES, delimiter=',', format='ascii.basic')

In [ ]:
# slice earthquake times to include only those in January, 
# then compress times so they all happen in a ten second period
jan =  q_table[np.where(Time(q_table['time']).jd < Time('2010-01-15', format='iso').jd)]
factor = 20/(31*24*60*60)
start = Time('2010-01-01', format='iso')
for i, val in enumerate(jan['time']):
    new = start.jd + (Time(val) - start).jd * factor
    jan['time'][i] = Time(new, format='jd').iso

In [ ]:
# slice locations to the area around japan
lons = np.intersect1d(np.where(jan['longitude'] > 135)[0],
                      np.where(jan['longitude'] < 165)[0])
lats = np.intersect1d(np.where(jan['latitude'] > 15)[0],
                      np.where(jan['latitude'] < 45)[0])
japan = jan[np.intersect1d(lons, lats)]
japan.remove_column('place') # causes parsing issues

In [ ]:
# whip up some test tables
j1 = japan.copy(); j20 = japan.copy(); j21 = japan.copy()
j22 = japan.copy(); j3 = japan.copy()

j1['time'] = Time(japan['time']) # astropy Times
j20['time'] = [t.datetime for t in j1['time']] # unaware datetimes
j21['time'] = [pytz.utc.localize(t) for t in j20['time']] # UTC datetimes
j22['time'] = [t.astimezone(pytz.timezone('America/Santiago'))
               for t in j21['time']] # non-UTC datetimes
j3['time'] = [t.isoformat() for t in j20['time']] # iso strings

Back to `time_test.py`

In [ ]:
wwt = WWTJupyterWidget()
wwt

In [ ]:
wwt.set_view('solar system')

In [ ]:
# center on Earth after shifting to proper scale
wwt.center_on_coordinates(SkyCoord(0,0,unit=u.deg), fov=.0001 * u.deg)
wwt.solar_system.track_object('Earth')

In [ ]:
# add the earthquake layer
jp_lay = wwt.layers.add_table_layer(table=j3, frame='Earth', lon_att='longitude',
                                    lat_att='latitude', size_scale=50, color='#ce1141',
                                    far_side_visible=True, time_series=True,
                                    time_att='time', time_decay=3*u.second)

In [ ]:
# rewind time to when the earthquakes took place
wwt.set_current_time(start)

In [ ]:
wwt.get_current_time()

In [ ]:
# if successful, remove jp_lay and test with the other jX test tables
jp_lay.remove()

The points should begin to pop up now.